In [363]:
#spark_df = spark.read.format("csv").option("header", "true").load("s3://xyz-source-raw/customer/customer_*.csv")

In [364]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Spark Job")\
        .enableHiveSupport()\
        .getOrCreate()

In [365]:
test_df = spark.read.csv("./final_customer_2023-2-10.csv",header=True)

In [366]:
test_df = test_df.withColumn("Date", F.col("Date").cast("date"))
test_df = test_df.withColumn("Customer_Joindate", F.col("Customer_Joindate").cast("date"))

In [367]:
test_df.printSchema()

root
 |-- Order_Id: string (nullable = true)
 |-- Customer_id: string (nullable = true)
 |-- Customer_name: string (nullable = true)
 |-- Customer_email: string (nullable = true)
 |-- Customer_address: string (nullable = true)
 |-- Customer_Joindate: date (nullable = true)
 |-- Product_id: string (nullable = true)
 |-- Product_name: string (nullable = true)
 |-- Product_category: string (nullable = true)
 |-- Manufacture_name: string (nullable = true)
 |-- Manufacture_address: string (nullable = true)
 |-- Product_Quantity: string (nullable = true)
 |-- Product_Price: string (nullable = true)
 |-- Store_id: string (nullable = true)
 |-- Store_name: string (nullable = true)
 |-- Manager_name: string (nullable = true)
 |-- Store_address: string (nullable = true)
 |-- Date: date (nullable = true)



In [368]:
Fact_table = test_df.select(['Order_id','Customer_id','Product_id','Date','Store_id','Product_Quantity','Product_Price'])

In [369]:
Fact_table.printSchema()

root
 |-- Order_id: string (nullable = true)
 |-- Product_id: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Store_id: string (nullable = true)
 |-- Product_Quantity: string (nullable = true)
 |-- Product_Price: string (nullable = true)



In [370]:
Fact_table.show()

+--------+----------+----------+--------+----------------+-------------+
|Order_id|Product_id|      Date|Store_id|Product_Quantity|Product_Price|
+--------+----------+----------+--------+----------------+-------------+
|       1|    85123A|2022-02-10|       1|               3|          100|
|       1|     71053|2022-02-10|       1|               3|          150|
|       1|    84406B|2022-02-10|       1|               2|          899|
|       2|    84029G|2022-02-10|       2|               3|           20|
|       3|    84029E|2022-02-10|       2|               1|           30|
|       4|     22752|2022-02-10|       2|               1|           20|
|       5|     21730|2022-02-10|       2|               3|           40|
|       6|     22633|2022-02-10|       3|               4|           50|
|       6|     84879|2022-02-10|       3|               5|           30|
|       7|     22633|2022-02-10|       3|               3|           50|
|       8|     84879|2022-02-10|       3|          

In [371]:
Fact_table = Fact_table.withColumn("Amount",Fact_table['Product_Quantity']*Fact_table['Product_Price'])

In [372]:
from pyspark.sql import functions as F

In [373]:
Fact_table = Fact_table.withColumn("Date_id",F.date_format(F.col("date"), "yyyyMMdd"))

In [374]:
Fact_table = Fact_table.select(['Order_id','Product_id','Date_id','Store_id','Product_Quantity','Amount'])
Fact_table = Fact_table.withColumnRenamed("Product_Quantity",'Quantity')
Fact_table.show(5)

+--------+----------+--------+--------+--------+------+
|Order_id|Product_id| Date_id|Store_id|Quantity|Amount|
+--------+----------+--------+--------+--------+------+
|       1|    85123A|20220210|       1|       3| 300.0|
|       1|     71053|20220210|       1|       3| 450.0|
|       1|    84406B|20220210|       1|       2|1798.0|
|       2|    84029G|20220210|       2|       3|  60.0|
|       3|    84029E|20220210|       2|       1|  30.0|
+--------+----------+--------+--------+--------+------+
only showing top 5 rows



In [375]:
Customer_dim = test_df.select(['Customer_id','Customer_name','Customer_email','Customer_address','Customer_Joindate'])

In [376]:
Customer_dim = Customer_dim.withColumn("First_Name", F.split(F.col('Customer_name'), ' ').getItem(0))
Customer_dim = Customer_dim.withColumn("Last_Name", F.split(F.col('Customer_name'), ' ').getItem(1))
Customer_dim = Customer_dim.withColumnRenamed("Customer_email",'Email_Address')


In [377]:
Customer_dim = Customer_dim.withColumn("Address", F.split(F.col('Customer_address'), ',').getItem(0))
Customer_dim = Customer_dim.withColumn("City", F.split(F.col('Customer_address'), ',').getItem(1))
Customer_dim = Customer_dim.withColumn("State", F.split(F.col('Customer_address'), ',').getItem(2))
Customer_dim = Customer_dim.withColumn("Zipcode", F.split(F.col('Customer_address'), ',').getItem(3))

In [379]:
Customer_dim = Customer_dim.select(
    F.col('Customer_id'),
    F.col('First_Name'),
    F.col('Last_Name'),
    F.col('Email_Address'),
    F.col('Address'),
    F.trim(F.col('City')).alias('City'),
   F.trim(F.col('State')).alias('State'),
    F.col('Zipcode'),
    F.col('Customer_Joindate').alias('join_date')
).dropDuplicates(subset=['Customer_id','First_Name']).show()

+-----------+----------+---------+--------------------+--------------------+-------------+-----+-------+
|Customer_id|First_Name|Last_Name|       Email_Address|             Address|         City|State|Zipcode|
+-----------+----------+---------+--------------------+--------------------+-------------+-----+-------+
|          1|    Hayley|   Austin| zekyrabe@finews.biz|   44 E. West Street|      Ashland|   OH|  44805|
|          2|   Michael|    Baker|  dypanoho@lyft.live|131 Iroquois Street |    Southgate|   FL|  04403|
|          3|     Carol|Jefferson|  sizunyxy@decabg.eu|MI 48195 8779 Win...|Fuquay Varina|   NC|  27526|
|          4|  Benjamin|    White|quburuha@ema-sofi...|    611 Penn Street |  Long Branch|   NJ|  07740|
|          5|     Jason|    Weber|    lyhevi@lyft.live|7075 Princess Street|       Linden|   NJ|  07036|
|          6|    Jordan|    Kelly| balukajo@finews.biz|      7459 Gulf Lane|      Raeford|   NC|  28376|
|          7|  Brittany|  Goodwin|gibumuba@lyricspa...|

In [380]:
Product_dim = test_df.select(
    ['Product_id','Product_name','Product_category','Manufacture_name','Manufacture_address']
)

In [381]:
Product_dim = Product_dim.withColumn("Address", F.split(F.col('Manufacture_address'), ',').getItem(0))
Product_dim = Product_dim.withColumn("City", F.split(F.col('Manufacture_address'), ',').getItem(1))
Product_dim = Product_dim.withColumn("State", F.split(F.col('Manufacture_address'), ',').getItem(2))
Product_dim = Product_dim.withColumn("Zipcode", F.split(F.col('Manufacture_address'), ',').getItem(3))

In [382]:
Product_dim = Product_dim.select(
    F.col('Product_id'),
    F.col('Product_name'),
    F.col('Product_category'),
    F.col('Manufacture_name'),
    F.col('Address'),
    F.trim(F.col('City')).alias('City'),
   F.trim(F.col('State')).alias('State'),
    F.col('Zipcode')
).dropDuplicates(subset=['Product_id','Product_name']).show()

+----------+--------------------+----------------+----------------+--------------------+------------+-----+-------+
|Product_id|        Product_name|Product_category|Manufacture_name|             Address|        City|State|Zipcode|
+----------+--------------------+----------------+----------------+--------------------+------------+-----+-------+
|     21730|GLASS STAR FROSTE...|         Kitchen|    Best Kitchen|  3862 Almond Avenue|     Fremont|   CA|  94538|
|     22633|HAND WARMER UNION...|           Cloth|          Unique|17372 Crestview Road|Fayetteville|   AR|  72701|
|     22752|SET 7 BABUSHKA NE...|         Kitchen|    Best Kitchen|  3862 Almond Avenue|     Fremont|   CA|  94538|
|     71053|           Apple Pen|            Tech|           Apple|463 Pecan Tree Drive|  Montgomery|   AL|  36109|
|    84029E|RED WOOLLY HOTTIE...|         Kitchen|    Best Kitchen|  3862 Almond Avenue|     Fremont|   CA|  94538|
|    84029G|KNITTED UNION FLA...|         Kitchen|    Best Kitchen|  386

In [383]:
Store_dim = test_df.select(
    ['Store_id','Store_name','Manager_name','Store_address']
)

In [384]:
Store_dim = Store_dim.withColumn("Address", F.split(F.col('Store_address'), ',').getItem(0))
Store_dim = Store_dim.withColumn("City", F.split(F.col('Store_address'), ',').getItem(1))
Store_dim = Store_dim.withColumn("State", F.split(F.col('Store_address'), ',').getItem(2))
Store_dim = Store_dim.withColumn("Zipcode", F.split(F.col('Store_address'), ',').getItem(3))

In [385]:
Store_dim = Store_dim.select(
    F.col('Store_id'),
    F.col('Store_name'),
    F.col('Manager_name'),
    F.col('Address'),
    F.trim(F.col('City')).alias('City'),
   F.trim(F.col('State')).alias('State'),
    F.col('Zipcode')
).dropDuplicates(subset=['Store_id','Store_name']).show()

+--------+------------------+------------+------------------+-----------+-----+-------+
|Store_id|        Store_name|Manager_name|           Address|       City|State|Zipcode|
+--------+------------------+------------+------------------+-----------+-----+-------+
|       1|      BestMai - NJ|       Afaka|       125 18th St|Jersey City|   NJ|  07310|
|       2|       BestMai -NY|        Dogg|      625 Broadway|   New York|   NY|  10012|
|       3|BestMai - Flushing|        Chen|13107 40th Rd C300|     Queens|   NY|  11354|
+--------+------------------+------------+------------------+-----------+-----+-------+

